In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn import neighbors
from sklearn.metrics import accuracy_score

In [2]:
world_data = pd.read_csv("world.csv")
life_data = pd.read_csv("life.csv")

In [3]:
world_data

,Country Name,Time,Country Code,Access to electricity (% of population) [EG.ELC.ACCS.ZS],Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD],Age dependency ratio (% of working-age population) [SP.POP.DPND],"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total) [SH.DTH.COMM.ZS]",Current health expenditure per capita (current US$) [SH.XPD.CHEX.PC.CD],"Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]",Fixed broadband subscriptions (per 100 people) [IT.NET.BBND.P2],...,Individuals using the Internet (% of population) [IT.NET.USER.ZS],Lifetime risk of maternal death (%) [SH.MMR.RISK.ZS],People using at least basic drinking water services (% of population) [SH.H2O.BASW.ZS],"People using at least basic drinking water services, rural (% of rural population) [SH.H2O.BASW.RU.ZS]","People using at least basic drinking water services, urban (% of urban population) [SH.H2O.BASW.UR.ZS]","People using at least basic sanitation services, urban (% of urban population) [SH.STA.BASS.UR.ZS]",Prevalence of anemia among children (% of children under 5) [SH.ANM.CHLD.ZS],Secure Internet servers (per 1 million people) [IT.NET.SECR.P6],"Self-employed, female (% of female employment) (modeled ILO estimate) [SL.EMP.SELF.FE.ZS]","Wage and salaried workers, female (% of female employment) (modeled ILO estimate) [SL.EMP.WORK.FE.ZS]"
0,Afghanistan,2016,AFG,97.7,465.8415828,87.94179015,36.4,61.45128632,4.8,0.024873504,...,11.2,3.330913861,64.28648363,54.84053165,92.59413481,59.9219787,46.4,12.03963652,93.58699799,6.413000107
1,Albania,2016,ALB,100,3776.111694,45.64503371,2.9,..,1.659,9.228640975,...,66.3634447,0.027187058,91.02037015,89.87517388,91.8354214,98.28801438,24.9,144.6402612,56.6230011,43.3769989
2,Algeria,2016,DZA,99.9923172,3284.470252,54.18401383,14.8,260.4223938,3.052,7.051711073,...,42.94552688,0.371412332,93.51585098,88.69095651,95.44293244,89.67236867,30,7.619958115,25.64599991,74.35399628
3,American Samoa,2016,ASM,..,..,..,..,..,..,..,...,..,..,99.69643741,..,..,..,..,179.4011589,..,..
4,Andorra,2016,AND,100,..,..,..,3843.702881,..,42.03267915,...,97.93063712,..,100,100,100,100,12.8,2833.22768,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,Virgin Islands (U.S.),2016,VIR,100,..,61.9053356,..,..,2.09,..,...,59.6083158,..,98.71826738,..,..,..,..,837.1314296,8.711000443,91.28900146
259,West Bank and Gaza,2016,PSE,100,..,74.34466371,..,..,3.844,6.917772552,...,61.17838483,0.124497152,96.47566794,95.44729645,96.80707586,97.19336735,26.4,108.0811745,29.25499916,70.74500275
260,"Yemen, Rep.",2016,YEM,76.06681061,1272.615878,76.0248698,28.7,..,3.994,1.016699372,...,24.57920836,0.700948502,63.32203386,54.7044139,79.0521,87.85827458,83.5,1.435501272,68.31900024,31.68099976
261,Zambia,2016,ZMB,35.21650696,972.1824246,91.97184156,60.6,56.54418182,4.814,0.194237673,...,25.50657885,1.136339587,59.2385641,41.17602711,83.73820865,36.23562282,54.2,8.861181164,88.56300354,11.43700027


In [4]:
life_data

,Country,Country Code,Year,Life expectancy at birth (years)
0,Afghanistan,AFG,2016,Low
1,Albania,ALB,2016,High
2,Algeria,DZA,2016,High
3,Angola,AGO,2016,Low
4,Antigua and Barbuda,ATG,2016,Medium
...,...,...,...,...
178,Venezuela (Bolivarian Republic of),VEN,2016,Medium
179,Viet Nam,VNM,2016,High
180,Yemen,YEM,2016,Low
181,Zambia,ZMB,2016,Low


In [5]:
# Merge and sort data on 'Country Code', then replace '..' with NaN
join_data = world_data.merge(life_data, on = 'Country Code')
join_data.drop(['Country','Year'], axis = 1, inplace = True)
join_data.sort_values('Country Code', ascending = True, inplace = True)
join_data.replace(r'..', np.nan, inplace = True)

In [6]:
join_data

,Country Name,Time,Country Code,Access to electricity (% of population) [EG.ELC.ACCS.ZS],Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD],Age dependency ratio (% of working-age population) [SP.POP.DPND],"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total) [SH.DTH.COMM.ZS]",Current health expenditure per capita (current US$) [SH.XPD.CHEX.PC.CD],"Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]",Fixed broadband subscriptions (per 100 people) [IT.NET.BBND.P2],...,Lifetime risk of maternal death (%) [SH.MMR.RISK.ZS],People using at least basic drinking water services (% of population) [SH.H2O.BASW.ZS],"People using at least basic drinking water services, rural (% of rural population) [SH.H2O.BASW.RU.ZS]","People using at least basic drinking water services, urban (% of urban population) [SH.H2O.BASW.UR.ZS]","People using at least basic sanitation services, urban (% of urban population) [SH.STA.BASS.UR.ZS]",Prevalence of anemia among children (% of children under 5) [SH.ANM.CHLD.ZS],Secure Internet servers (per 1 million people) [IT.NET.SECR.P6],"Self-employed, female (% of female employment) (modeled ILO estimate) [SL.EMP.SELF.FE.ZS]","Wage and salaried workers, female (% of female employment) (modeled ILO estimate) [SL.EMP.WORK.FE.ZS]",Life expectancy at birth (years)
0,Afghanistan,2016,AFG,97.7,465.8415828,87.94179015,36.4,61.45128632,4.8,0.024873504,...,3.330913861,64.28648363,54.84053165,92.59413481,59.9219787,46.4,12.03963652,93.58699799,6.413000107,Low
3,Angola,2016,AGO,40.62452698,1949.11468,97.29458812,63.4,95.22080231,5.686,0.293747187,...,1.501205426,55.08427629,27.07967323,70.73522869,63.33675598,50.9,8.321058616,82.67299652,17.32699966,Low
1,Albania,2016,ALB,100,3776.111694,45.64503371,2.9,NaN,1.659,9.228640975,...,0.027187058,91.02037015,89.87517388,91.8354214,98.28801438,24.9,144.6402612,56.6230011,43.3769989,High
172,United Arab Emirates,2016,ARE,100,35465.40518,18.16490433,6.5,1310.923096,1.491,13.86163628,...,0.005711886,97.40736883,NaN,NaN,NaN,27.2,938.6837703,1.307999969,98.69200134,High
5,Argentina,2016,ARG,99.9668808,11006.02931,56.10227058,15.9,959.2915649,2.288,16.60166322,...,0.095582587,99.07837465,92.98366005,99.63532288,95.85623215,22.2,735.0247651,20.79500008,79.20500183,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Samoa,2016,WSM,99.78393555,3485.940268,76.19806715,12,227.1501312,3.976,1.114966458,...,0.174042785,97.30364572,96.68422131,100,98.5081203,43,149.0734315,27.5909996,72.40899658,High
180,"Yemen, Rep.",2016,YEM,76.06681061,1272.615878,76.0248698,28.7,NaN,3.994,1.016699372,...,0.700948502,63.32203386,54.7044139,79.0521,87.85827458,83.5,1.435501272,68.31900024,31.68099976,Low
149,South Africa,2016,ZAF,84.2,4252.481244,52.28947458,39.6,428.296875,2.457,2.04735491,...,0.313375078,92.27031216,79.82383736,98.8723329,76.00077975,36.8,917.6449631,12.33699989,87.66300201,Low
181,Zambia,2016,ZMB,35.21650696,972.1824246,91.97184156,60.6,56.54418182,4.814,0.194237673,...,1.136339587,59.2385641,41.17602711,83.73820865,36.23562282,54.2,8.861181164,88.56300354,11.43700027,Low


In [7]:
# Find all features of data
features = [column for column in join_data.columns.tolist() if column not in ['Country Name', 'Country Code', 'Time', 'Life expectancy at birth (years)']]
print("List of Features: \n" + "\n".join(features))
print("\nTotal Number of Features = " + str(len(features)))

List of Features: 
Access to electricity (% of population) [EG.ELC.ACCS.ZS]
Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD]
Age dependency ratio (% of working-age population) [SP.POP.DPND]
Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total) [SH.DTH.COMM.ZS]
Current health expenditure per capita (current US$) [SH.XPD.CHEX.PC.CD]
Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]
Fixed broadband subscriptions (per 100 people) [IT.NET.BBND.P2]
Fixed telephone subscriptions (per 100 people) [IT.MLT.MAIN.P2]
GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD]
GNI per capita, Atlas method (current US$) [NY.GNP.PCAP.CD]
Individuals using the Internet (% of population) [IT.NET.USER.ZS]
Lifetime risk of maternal death (%) [SH.MMR.RISK.ZS]
People using at least basic drinking water services (% of population) [SH.H2O.BASW.ZS]
People using at least basic drinking water services, rural (% of rural population) [SH.H2O

In [8]:
# Split data into features and class feature
X = join_data[features]
X

,Access to electricity (% of population) [EG.ELC.ACCS.ZS],Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD],Age dependency ratio (% of working-age population) [SP.POP.DPND],"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total) [SH.DTH.COMM.ZS]",Current health expenditure per capita (current US$) [SH.XPD.CHEX.PC.CD],"Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]",Fixed broadband subscriptions (per 100 people) [IT.NET.BBND.P2],Fixed telephone subscriptions (per 100 people) [IT.MLT.MAIN.P2],GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD],"GNI per capita, Atlas method (current US$) [NY.GNP.PCAP.CD]",Individuals using the Internet (% of population) [IT.NET.USER.ZS],Lifetime risk of maternal death (%) [SH.MMR.RISK.ZS],People using at least basic drinking water services (% of population) [SH.H2O.BASW.ZS],"People using at least basic drinking water services, rural (% of rural population) [SH.H2O.BASW.RU.ZS]","People using at least basic drinking water services, urban (% of urban population) [SH.H2O.BASW.UR.ZS]","People using at least basic sanitation services, urban (% of urban population) [SH.STA.BASS.UR.ZS]",Prevalence of anemia among children (% of children under 5) [SH.ANM.CHLD.ZS],Secure Internet servers (per 1 million people) [IT.NET.SECR.P6],"Self-employed, female (% of female employment) (modeled ILO estimate) [SL.EMP.SELF.FE.ZS]","Wage and salaried workers, female (% of female employment) (modeled ILO estimate) [SL.EMP.WORK.FE.ZS]"
0,97.7,465.8415828,87.94179015,36.4,61.45128632,4.8,0.024873504,0.322730963,571.0737745,570,11.2,3.330913861,64.28648363,54.84053165,92.59413481,59.9219787,46.4,12.03963652,93.58699799,6.413000107
3,40.62452698,1949.11468,97.29458812,63.4,95.22080231,5.686,0.293747187,1.055709859,3530.309423,3770,13,1.501205426,55.08427629,27.07967323,70.73522869,63.33675598,50.9,8.321058616,82.67299652,17.32699966
1,100,3776.111694,45.64503371,2.9,NaN,1.659,9.228640975,8.614077281,4681.840039,4320,66.3634447,0.027187058,91.02037015,89.87517388,91.8354214,98.28801438,24.9,144.6402612,56.6230011,43.3769989
172,100,35465.40518,18.16490433,6.5,1310.923096,1.491,13.86163628,24.41847969,41045.11112,40570,90.60000732,0.005711886,97.40736883,NaN,NaN,NaN,27.2,938.6837703,1.307999969,98.69200134
5,99.9668808,11006.02931,56.10227058,15.9,959.2915649,2.288,16.60166322,23.36292528,10239.48135,12220,70.96898082,0.095582587,99.07837465,92.98366005,99.63532288,95.85623215,22.2,735.0247651,20.79500008,79.20500183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,99.78393555,3485.940268,76.19806715,12,227.1501312,3.976,1.114966458,4.975454288,3824.501891,4110,29.41173011,0.174042785,97.30364572,96.68422131,100,98.5081203,43,149.0734315,27.5909996,72.40899658
180,76.06681061,1272.615878,76.0248698,28.7,NaN,3.994,1.016699372,4.291147948,765.6041721,NaN,24.57920836,0.700948502,63.32203386,54.7044139,79.0521,87.85827458,83.5,1.435501272,68.31900024,31.68099976
149,84.2,4252.481244,52.28947458,39.6,428.296875,2.457,2.04735491,8.046681051,7476.621012,5470,54,0.313375078,92.27031216,79.82383736,98.8723329,76.00077975,36.8,917.6449631,12.33699989,87.66300201
181,35.21650696,972.1824246,91.97184156,60.6,56.54418182,4.814,0.194237673,0.619716199,1652.284458,1380,25.50657885,1.136339587,59.2385641,41.17602711,83.73820865,36.23562282,54.2,8.861181164,88.56300354,11.43700027


In [9]:
y = join_data['Life expectancy at birth (years)']
y

0       Low
3       Low
1      High
172    High
5      High
       ... 
137    High
180     Low
149     Low
181     Low
182     Low
Name: Life expectancy at birth (years), Length: 183, dtype: object

In [10]:
# Split the dataset into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, test_size = 0.3, random_state = 200)

In [11]:
X_train.astype(float).describe().loc[['mean', '50%', '75%', 'max']]

,Access to electricity (% of population) [EG.ELC.ACCS.ZS],Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD],Age dependency ratio (% of working-age population) [SP.POP.DPND],"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total) [SH.DTH.COMM.ZS]",Current health expenditure per capita (current US$) [SH.XPD.CHEX.PC.CD],"Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]",Fixed broadband subscriptions (per 100 people) [IT.NET.BBND.P2],Fixed telephone subscriptions (per 100 people) [IT.MLT.MAIN.P2],GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD],"GNI per capita, Atlas method (current US$) [NY.GNP.PCAP.CD]",Individuals using the Internet (% of population) [IT.NET.USER.ZS],Lifetime risk of maternal death (%) [SH.MMR.RISK.ZS],People using at least basic drinking water services (% of population) [SH.H2O.BASW.ZS],"People using at least basic drinking water services, rural (% of rural population) [SH.H2O.BASW.RU.ZS]","People using at least basic drinking water services, urban (% of urban population) [SH.H2O.BASW.UR.ZS]","People using at least basic sanitation services, urban (% of urban population) [SH.STA.BASS.UR.ZS]",Prevalence of anemia among children (% of children under 5) [SH.ANM.CHLD.ZS],Secure Internet servers (per 1 million people) [IT.NET.SECR.P6],"Self-employed, female (% of female employment) (modeled ILO estimate) [SL.EMP.SELF.FE.ZS]","Wage and salaried workers, female (% of female employment) (modeled ILO estimate) [SL.EMP.WORK.FE.ZS]"
mean,81.937700,10068.005670,59.845307,23.247656,1003.793602,2.809573,11.702004,15.418202,13555.708443,12728.278689,48.527329,0.795386,86.166662,77.860163,93.358786,76.949143,37.425,2312.816783,44.850336,55.149664
50%,98.440967,3963.975616,54.294372,13.500000,296.938904,2.354000,6.191481,10.764395,5669.733946,5375.000000,48.052274,0.162758,93.734351,87.794938,97.412091,89.368273,31.900,119.659418,40.410999,59.589001
75%,100.000000,11636.940315,71.184224,40.300000,1052.516113,3.817750,19.031262,22.503886,15818.419445,14877.500000,75.043115,1.104440,99.095825,98.826595,99.627766,98.288014,48.900,1200.566740,75.041000,86.998001
max,100.000000,64069.585760,111.602727,65.300000,9835.963867,7.087000,45.059110,60.317690,110162.120800,82110.000000,98.240016,7.038785,100.000002,100.000000,100.000000,100.000000,86.200,35663.712330,98.912003,98.692001


In [12]:
X_train

,Access to electricity (% of population) [EG.ELC.ACCS.ZS],Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD],Age dependency ratio (% of working-age population) [SP.POP.DPND],"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total) [SH.DTH.COMM.ZS]",Current health expenditure per capita (current US$) [SH.XPD.CHEX.PC.CD],"Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]",Fixed broadband subscriptions (per 100 people) [IT.NET.BBND.P2],Fixed telephone subscriptions (per 100 people) [IT.MLT.MAIN.P2],GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD],"GNI per capita, Atlas method (current US$) [NY.GNP.PCAP.CD]",Individuals using the Internet (% of population) [IT.NET.USER.ZS],Lifetime risk of maternal death (%) [SH.MMR.RISK.ZS],People using at least basic drinking water services (% of population) [SH.H2O.BASW.ZS],"People using at least basic drinking water services, rural (% of rural population) [SH.H2O.BASW.RU.ZS]","People using at least basic drinking water services, urban (% of urban population) [SH.H2O.BASW.UR.ZS]","People using at least basic sanitation services, urban (% of urban population) [SH.STA.BASS.UR.ZS]",Prevalence of anemia among children (% of children under 5) [SH.ANM.CHLD.ZS],Secure Internet servers (per 1 million people) [IT.NET.SECR.P6],"Self-employed, female (% of female employment) (modeled ILO estimate) [SL.EMP.SELF.FE.ZS]","Wage and salaried workers, female (% of female employment) (modeled ILO estimate) [SL.EMP.WORK.FE.ZS]"
59,91.05812836,4987.601308,67.20863011,49.9,219.9850922,4.039,0.747955672,0.943585575,9419.856712,7080,48.05227375,1.095333321,85.62704687,55.05480828,89.57669523,48.65327072,62.5,22.41177604,40.4109993,59.5890007
33,100,10755.31922,45.30184992,8.1,1190.548584,1.713,15.95128317,18.53492447,14771.25765,13420,83.55858602,0.023301792,99.62218018,98.3252178,99.80878307,100,20,1237.18578,25.9829998,74.01699829
10,100,27169.17566,43.32397891,16.9,1837.258789,1.771,22.52480618,32.03979573,27370.20254,28910,80,0.125916474,98.8869605,NaN,NaN,NaN,29.5,1053.102286,8.201999664,91.79799652
25,23.65287399,414.3358957,91.56125413,56.3,40.95466232,5.353,0.066415118,0.406122225,663.8474288,600,13.96,1.829265196,48.26771688,36.04523782,79.48903443,40.13079418,86.2,1.823410423,90.68699646,9.312999725
23,100,24338.80432,39.24499889,7.8,686.4823608,1.915,8.604097189,17.67818009,31685.22545,33160,90,0.06090028,99.90000909,NaN,NaN,NaN,16.1,612.1962839,7.955999851,92.04399872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,100,13355.25137,50.89273244,5.4,1321.620239,1.63,28.91055035,16.56101029,21863.64165,17590,76.4811987,0.00602772,99.88016949,99.8175993,99.90265093,99.09274194,25.8,11991.38925,13.22999954,86.76999664
66,91.92584229,3494.706279,67.37000913,25.1,248.8022003,2.973,3.05130363,14.80291111,3100.829958,3800,34.50932609,0.320602352,93.72089205,89.66381373,97.72652989,79.37134783,36.5,36.18150088,47.38299942,52.61700058
11,100,19133.21509,29.4204093,6.9,1099.362305,2.033,16.29866067,19.62866954,22336.77993,21610,97.9999807,0.034427671,100,NaN,NaN,NaN,30.1,346.474343,3.118000031,96.88200378
102,99.96797943,7382.518121,32.63405994,8,1052.516113,1.947,6.163672,4.445103383,7631.471877,8070,59.09258997,0.125220073,99.23921181,99.85330875,98.27685084,99.1348876,39,212.4021846,35.01800156,64.98200226


In [13]:
print("Number of missing values of each column in the training data")
for name in X_train.columns:
    print(f"{name}: {X_train[name].isna().sum()}")

Number of missing values of each column in the training data
Access to electricity (% of population) [EG.ELC.ACCS.ZS]: 0
Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD]: 8
Age dependency ratio (% of working-age population) [SP.POP.DPND]: 1
Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total) [SH.DTH.COMM.ZS]: 0
Current health expenditure per capita (current US$) [SH.XPD.CHEX.PC.CD]: 7
Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]: 0
Fixed broadband subscriptions (per 100 people) [IT.NET.BBND.P2]: 4
Fixed telephone subscriptions (per 100 people) [IT.MLT.MAIN.P2]: 0
GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD]: 5
GNI per capita, Atlas method (current US$) [NY.GNP.PCAP.CD]: 6
Individuals using the Internet (% of population) [IT.NET.USER.ZS]: 1
Lifetime risk of maternal death (%) [SH.MMR.RISK.ZS]: 0
People using at least basic drinking water services (% of population) [SH.H2O.BASW.ZS]: 0
People us

In [14]:
# Impute with median
my_imputer = SimpleImputer(strategy = 'median')

imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))

imputed_X_train.columns = X_train.columns
imputed_X_test.columns = X_test.columns

In [15]:
imputed_X_train

,Access to electricity (% of population) [EG.ELC.ACCS.ZS],Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD],Age dependency ratio (% of working-age population) [SP.POP.DPND],"Cause of death, by communicable diseases and maternal, prenatal and nutrition conditions (% of total) [SH.DTH.COMM.ZS]",Current health expenditure per capita (current US$) [SH.XPD.CHEX.PC.CD],"Fertility rate, total (births per woman) [SP.DYN.TFRT.IN]",Fixed broadband subscriptions (per 100 people) [IT.NET.BBND.P2],Fixed telephone subscriptions (per 100 people) [IT.MLT.MAIN.P2],GDP per capita (constant 2010 US$) [NY.GDP.PCAP.KD],"GNI per capita, Atlas method (current US$) [NY.GNP.PCAP.CD]",Individuals using the Internet (% of population) [IT.NET.USER.ZS],Lifetime risk of maternal death (%) [SH.MMR.RISK.ZS],People using at least basic drinking water services (% of population) [SH.H2O.BASW.ZS],"People using at least basic drinking water services, rural (% of rural population) [SH.H2O.BASW.RU.ZS]","People using at least basic drinking water services, urban (% of urban population) [SH.H2O.BASW.UR.ZS]","People using at least basic sanitation services, urban (% of urban population) [SH.STA.BASS.UR.ZS]",Prevalence of anemia among children (% of children under 5) [SH.ANM.CHLD.ZS],Secure Internet servers (per 1 million people) [IT.NET.SECR.P6],"Self-employed, female (% of female employment) (modeled ILO estimate) [SL.EMP.SELF.FE.ZS]","Wage and salaried workers, female (% of female employment) (modeled ILO estimate) [SL.EMP.WORK.FE.ZS]"
0,91.058128,4987.601308,67.208630,49.9,219.985092,4.039,0.747956,0.943586,9419.856712,7080.0,48.052274,1.095333,85.627047,55.054808,89.576695,48.653271,62.5,22.411776,40.410999,59.589001
1,100.000000,10755.319220,45.301850,8.1,1190.548584,1.713,15.951283,18.534924,14771.257650,13420.0,83.558586,0.023302,99.622180,98.325218,99.808783,100.000000,20.0,1237.185780,25.983000,74.016998
2,100.000000,27169.175660,43.323979,16.9,1837.258789,1.771,22.524806,32.039796,27370.202540,28910.0,80.000000,0.125916,98.886961,87.794938,97.412091,89.368273,29.5,1053.102286,8.202000,91.797997
3,23.652874,414.335896,91.561254,56.3,40.954662,5.353,0.066415,0.406122,663.847429,600.0,13.960000,1.829265,48.267717,36.045238,79.489034,40.130794,86.2,1.823410,90.686996,9.313000
4,100.000000,24338.804320,39.244999,7.8,686.482361,1.915,8.604097,17.678180,31685.225450,33160.0,90.000000,0.060900,99.900009,87.794938,97.412091,89.368273,16.1,612.196284,7.956000,92.043999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,100.000000,13355.251370,50.892732,5.4,1321.620239,1.630,28.910550,16.561010,21863.641650,17590.0,76.481199,0.006028,99.880169,99.817599,99.902651,99.092742,25.8,11991.389250,13.230000,86.769997
124,91.925842,3494.706279,67.370009,25.1,248.802200,2.973,3.051304,14.802911,3100.829958,3800.0,34.509326,0.320602,93.720892,89.663814,97.726530,79.371348,36.5,36.181501,47.382999,52.617001
125,100.000000,19133.215090,29.420409,6.9,1099.362305,2.033,16.298661,19.628670,22336.779930,21610.0,97.999981,0.034428,100.000000,87.794938,97.412091,89.368273,30.1,346.474343,3.118000,96.882004
126,99.967979,7382.518121,32.634060,8.0,1052.516113,1.947,6.163672,4.445103,7631.471877,8070.0,59.092590,0.125220,99.239212,99.853309,98.276851,99.134888,39.0,212.402185,35.018002,64.982002


In [16]:
print("X_train after median imputation: \n", imputed_X_train.head())

X_train after median imputation: 
    Access to electricity (% of population) [EG.ELC.ACCS.ZS]  \
0                                          91.058128          
1                                         100.000000          
2                                         100.000000          
3                                          23.652874          
4                                         100.000000          

   Adjusted net national income per capita (current US$) [NY.ADJ.NNTY.PC.CD]  \
0                                        4987.601308                           
1                                       10755.319220                           
2                                       27169.175660                           
3                                         414.335896                           
4                                       24338.804320                           

   Age dependency ratio (% of working-age population) [SP.POP.DPND]  \
0                                 

In [17]:
# Normalize values of the data
scaler = preprocessing.StandardScaler().fit(imputed_X_train)

imputed_X_train = scaler.transform(imputed_X_train)
imputed_X_test = scaler.transform(imputed_X_test)

In [18]:
# Extract median, mean and variance of the training data
np.set_printoptions(suppress=True)

median = np.round(my_imputer.statistics_, 3)
mean = np.round(scaler.mean_, 3)
var = np.round(scaler.var_.astype(float), 3)

In [19]:
# Output the median, mean and variance of the data
task2 = {'feature': features, 'median': median, 'mean': mean, 'variance': var}
task2_data = pd.DataFrame(task2)
task2_data.set_index('feature', inplace = True)
task2_data.to_csv('task2a.csv')

In [20]:
# Perform KNearestNeighbor with 3 clusters
knn3 = neighbors.KNeighborsClassifier(n_neighbors=3)
knn3.fit(imputed_X_train, y_train)
knn3_y_pred = knn3.predict(imputed_X_test)
knn3_score = accuracy_score(y_test, knn3_y_pred)

In [21]:
# Perform KNearestNeighbor with 7 clusters
knn7 = neighbors.KNeighborsClassifier(n_neighbors=7)
knn7.fit(imputed_X_train, y_train)
knn7_y_pred = knn7.predict(imputed_X_test)
knn7_score = accuracy_score(y_test, knn7_y_pred)

In [22]:
# Perform DecisionTree with max depth equals 3
dt = DecisionTreeClassifier(random_state = 200, max_depth=3)
dt.fit(imputed_X_train, y_train)
dt_y_pred = dt.predict(imputed_X_test)
dt_score = accuracy_score(y_test, dt_y_pred)

In [23]:
print(f"Accuracy of decision tree: {dt_score:.3f}\nAccuracy of k-nn (k=3): {knn3_score:.3f}\nAccuracy of k-nn (k=7): {knn7_score:.3f}")

Accuracy of decision tree: 0.709
Accuracy of k-nn (k=3): 0.673
Accuracy of k-nn (k=7): 0.727
